<a href="https://colab.research.google.com/github/awitz23/Seminararbeit_CS-GO_Spieltheorie/blob/main/Datei_zur_Kaufentscheidung/Kaufentscheidung_Code_mit_ID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installationen

In [ ]:
pip install Pillow==9.0.0

In [ ]:
#shell notwendig, da awpy golang benötigt

%%shell
add-apt-repository -y ppa:longsleep/golang-backports
apt -y update
apt -y install golang-go

pip install awpy


In [ ]:
!pip install nashpy
import nashpy as nash

import pandas as pd
import numpy as np
from awpy import *
from awpy.data import NAV_CSV
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')
import plotly.graph_objects as go
import pandas as pd
import os
dir = '/content/drive/MyDrive/2022_Dust2'

In [ ]:
#Initialisiere ein leeres Dataframe für spätere Nutzung
df_strategy=pd.DataFrame()

#Lege Pfad fest, aus dem die Dateien gelesen werden
dir = '/content/drive/MyDrive/2022_Dust2'


#Kaufentscheidung - Parsing

In [ ]:
#Parsing

#Initialisiere Listen
CT_Buy = []
T_Buy = []
Winner_list = []
GameRound_list = []
Game_id =[]

#Gehe den vorher definierten Pfad durch
for file in os.listdir(dir):
  #gehe für jedes Element, das auf .dem endet folgende Befehle durch
  if file.endswith(".dem"):
    
    #Starte das Einlesen und speichert die Daten als DataFrame 
    p = DemoParser(demofile="/content/drive/MyDrive/CS:GO_Dust2/2018_Dust2/"+file, parse_rate=128)
    df = p.parse(return_type="df")
    id = ntpath.basename("/content/drive/MyDrive/CS:GO_Dust2/2018_Dust2/"+file)
    year = "18"
    #erstelle temporäre Liste für eine Datei
    #hat für jede Datei andere Werte 
    CT_buy_temp = df["rounds"]["ctBuyType"].tolist()
    T_buy_temp = df["rounds"]["tBuyType"].tolist()
    Winner_list_temp = df["rounds"]["winningSide"].tolist()
    Gameround_list_temp = df["rounds"]["roundNum"].tolist()
    Game_id_temp = []
    i=1
    while i <= df["playerFrames"]["roundNum"].max():
      Game_id_temp.append(id+year)
      i=i+1
    #Speichere die temporären Listen in einer gesamten Liste
    CT_Buy.extend(CT_buy_temp)
    T_Buy.extend(T_buy_temp)
    Winner_list.extend(Winner_list_temp)
    GameRound_list.extend(Gameround_list_temp)
    Game_id.extend(Game_id_temp)
    #Wandle die Daten in ein DataFrame um
    Buy_Dec = pd.DataFrame({'Round':GameRound_list, "T-Buy":T_Buy, "CT-Buy":CT_Buy, "Winner":Winner_list, "ID":Game_id})

#Kaufentscheidung - Prepocessing

In [ ]:
#Aufteilung des großen DF in 16 kleine
#Werte der T-Seite

pi_F_B_F_B = Buy_Dec[(Buy_Dec['T-Buy'] == "Full Buy") & (Buy_Dec['CT-Buy'] == "Full Buy") & (Buy_Dec['Winner'] == "T")]
pi_F_B_S_B = Buy_Dec[(Buy_Dec['T-Buy'] == "Full Buy") & (Buy_Dec['CT-Buy'] == "Semi Buy") & (Buy_Dec['Winner'] == "T")]
pi_F_B_F_E = Buy_Dec[(Buy_Dec['T-Buy'] == "Full Buy") & (Buy_Dec['CT-Buy'] == "Full Eco") & (Buy_Dec['Winner'] == "T")]
pi_F_B_S_E = Buy_Dec[(Buy_Dec['T-Buy'] == "Full Buy") & (Buy_Dec['CT-Buy'] == "Semi Eco") & (Buy_Dec['Winner'] == "T")]
pi_S_B_F_B = Buy_Dec[(Buy_Dec['T-Buy'] == "Semi Buy") & (Buy_Dec['CT-Buy'] == "Full Buy") & (Buy_Dec['Winner'] == "T")]
pi_S_B_S_B = Buy_Dec[(Buy_Dec['T-Buy'] == "Semi Buy") & (Buy_Dec['CT-Buy'] == "Semi Buy") & (Buy_Dec['Winner'] == "T")]
pi_S_B_F_E = Buy_Dec[(Buy_Dec['T-Buy'] == "Semi Buy") & (Buy_Dec['CT-Buy'] == "Full Eco") & (Buy_Dec['Winner'] == "T")]
pi_S_B_S_E = Buy_Dec[(Buy_Dec['T-Buy'] == "Semi Buy") & (Buy_Dec['CT-Buy'] == "Semi Eco") & (Buy_Dec['Winner'] == "T")]
pi_F_E_F_B = Buy_Dec[(Buy_Dec['T-Buy'] == "Full Eco") & (Buy_Dec['CT-Buy'] == "Full Buy") & (Buy_Dec['Winner'] == "T")]
pi_F_E_S_B = Buy_Dec[(Buy_Dec['T-Buy'] == "Full Eco") & (Buy_Dec['CT-Buy'] == "Semi Buy") & (Buy_Dec['Winner'] == "T")]
pi_F_E_F_E = Buy_Dec[(Buy_Dec['T-Buy'] == "Full Eco") & (Buy_Dec['CT-Buy'] == "Full Eco") & (Buy_Dec['Winner'] == "T")]
pi_F_E_S_E = Buy_Dec[(Buy_Dec['T-Buy'] == "Full Eco") & (Buy_Dec['CT-Buy'] == "Semi Eco") & (Buy_Dec['Winner'] == "T")]
pi_S_E_F_B = Buy_Dec[(Buy_Dec['T-Buy'] == "Semi Eco") & (Buy_Dec['CT-Buy'] == "Full Buy") & (Buy_Dec['Winner'] == "T")]
pi_S_E_S_B = Buy_Dec[(Buy_Dec['T-Buy'] == "Semi Eco") & (Buy_Dec['CT-Buy'] == "Semi Buy") & (Buy_Dec['Winner'] == "T")]
pi_S_E_F_E = Buy_Dec[(Buy_Dec['T-Buy'] == "Semi Eco") & (Buy_Dec['CT-Buy'] == "Full Eco") & (Buy_Dec['Winner'] == "T")]
pi_S_E_S_E = Buy_Dec[(Buy_Dec['T-Buy'] == "Semi Eco") & (Buy_Dec['CT-Buy'] == "Semi Eco") & (Buy_Dec['Winner'] == "T")]

In [ ]:
#Werte der CT-Seite
CT_pi_F_B_F_B = Buy_Dec[(Buy_Dec['T-Buy'] == "Full Buy") & (Buy_Dec['CT-Buy'] == "Full Buy") & (Buy_Dec['Winner'] == "CT")]
CT_pi_F_B_S_B = Buy_Dec[(Buy_Dec['T-Buy'] == "Full Buy") & (Buy_Dec['CT-Buy'] == "Semi Buy") & (Buy_Dec['Winner'] == "CT")]
CT_pi_F_B_F_E = Buy_Dec[(Buy_Dec['T-Buy'] == "Full Buy") & (Buy_Dec['CT-Buy'] == "Full Eco") & (Buy_Dec['Winner'] == "CT")]
CT_pi_F_B_S_E = Buy_Dec[(Buy_Dec['T-Buy'] == "Full Buy") & (Buy_Dec['CT-Buy'] == "Semi Eco") & (Buy_Dec['Winner'] == "CT")]
CT_pi_S_B_F_B = Buy_Dec[(Buy_Dec['T-Buy'] == "Semi Buy") & (Buy_Dec['CT-Buy'] == "Full Buy") & (Buy_Dec['Winner'] == "CT")]
CT_pi_S_B_S_B = Buy_Dec[(Buy_Dec['T-Buy'] == "Semi Buy") & (Buy_Dec['CT-Buy'] == "Semi Buy") & (Buy_Dec['Winner'] == "CT")]
CT_pi_S_B_F_E = Buy_Dec[(Buy_Dec['T-Buy'] == "Semi Buy") & (Buy_Dec['CT-Buy'] == "Full Eco") & (Buy_Dec['Winner'] == "CT")]
CT_pi_S_B_S_E = Buy_Dec[(Buy_Dec['T-Buy'] == "Semi Buy") & (Buy_Dec['CT-Buy'] == "Semi Eco") & (Buy_Dec['Winner'] == "CT")]
CT_pi_F_E_F_B = Buy_Dec[(Buy_Dec['T-Buy'] == "Full Eco") & (Buy_Dec['CT-Buy'] == "Full Buy") & (Buy_Dec['Winner'] == "CT")]
CT_pi_F_E_S_B = Buy_Dec[(Buy_Dec['T-Buy'] == "Full Eco") & (Buy_Dec['CT-Buy'] == "Semi Buy") & (Buy_Dec['Winner'] == "CT")]
CT_pi_F_E_F_E = Buy_Dec[(Buy_Dec['T-Buy'] == "Full Eco") & (Buy_Dec['CT-Buy'] == "Full Eco") & (Buy_Dec['Winner'] == "CT")]
CT_pi_F_E_S_E = Buy_Dec[(Buy_Dec['T-Buy'] == "Full Eco") & (Buy_Dec['CT-Buy'] == "Semi Eco") & (Buy_Dec['Winner'] == "CT")]
CT_pi_S_E_F_B = Buy_Dec[(Buy_Dec['T-Buy'] == "Semi Eco") & (Buy_Dec['CT-Buy'] == "Full Buy") & (Buy_Dec['Winner'] == "CT")]
CT_pi_S_E_S_B = Buy_Dec[(Buy_Dec['T-Buy'] == "Semi Eco") & (Buy_Dec['CT-Buy'] == "Semi Buy") & (Buy_Dec['Winner'] == "CT")]
CT_pi_S_E_F_E = Buy_Dec[(Buy_Dec['T-Buy'] == "Semi Eco") & (Buy_Dec['CT-Buy'] == "Full Eco") & (Buy_Dec['Winner'] == "CT")]
CT_pi_S_E_S_E = Buy_Dec[(Buy_Dec['T-Buy'] == "Semi Eco") & (Buy_Dec['CT-Buy'] == "Semi Eco") & (Buy_Dec['Winner'] == "CT")]

In [ ]:
#Gesamtzahl der Ereignisse

full_pi_F_B_F_B = pi_F_B_F_B.shape[0] + CT_pi_F_B_F_B.shape[0]
full_pi_F_B_S_B = pi_F_B_S_B.shape[0] + CT_pi_F_B_S_B.shape[0]
full_pi_F_B_F_E = pi_F_B_F_E.shape[0] + CT_pi_F_B_F_E.shape[0]
full_pi_F_B_S_E = pi_F_B_S_E.shape[0] + CT_pi_F_B_S_E.shape[0]
full_pi_S_B_F_B = pi_S_B_F_B.shape[0] + CT_pi_S_B_F_B.shape[0]
full_pi_S_B_S_B = pi_S_B_S_B.shape[0] + CT_pi_S_B_S_B.shape[0]
full_pi_S_B_F_E = pi_S_B_F_E.shape[0] + CT_pi_S_B_F_E.shape[0]
full_pi_S_B_S_E = pi_S_B_S_E.shape[0] + CT_pi_S_B_S_E.shape[0]
full_pi_F_E_F_B = pi_F_E_F_B.shape[0] + CT_pi_F_E_F_B.shape[0]
full_pi_F_E_S_B = pi_F_E_S_B.shape[0] + CT_pi_F_E_S_B.shape[0]
full_pi_F_E_F_E = pi_F_E_F_E.shape[0] + CT_pi_F_E_F_E.shape[0]
full_pi_F_E_S_E = pi_F_E_S_E.shape[0] + CT_pi_F_E_S_E.shape[0]
full_pi_S_E_F_B = pi_S_E_F_B.shape[0] + CT_pi_S_E_F_B.shape[0]
full_pi_S_E_S_B = pi_S_E_S_B.shape[0] + CT_pi_S_E_S_B.shape[0]
full_pi_S_E_F_E = pi_S_E_F_E.shape[0] + CT_pi_S_E_F_E.shape[0]
full_pi_S_E_S_E = pi_S_E_S_E.shape[0] + CT_pi_S_E_S_E.shape[0]


#Setze alle 0 Werte auf 1, damit Berechnung der Gewinnwahrscheinlichkeit nicht auf Fehler läuft
#Wird nur zum dividieren benötigt

if full_pi_F_B_F_B == 0:
  full_pi_F_B_F_B = 1
else:
  full_pi_F_B_F_B = full_pi_F_B_F_B

if full_pi_F_B_S_B == 0:
  full_pi_F_B_S_B = 1
else:
  full_pi_F_B_S_B = full_pi_F_B_S_B

if full_pi_F_B_F_E == 0:
  full_pi_F_B_F_E = 1
else:
  full_pi_F_B_F_E = full_pi_F_B_F_E

if full_pi_F_B_S_E == 0:
  full_pi_F_B_S_E = 1
else:
  full_pi_F_B_S_E = full_pi_F_B_S_E

if full_pi_S_B_F_B == 0:
  full_pi_S_B_F_B = 1
else:
  full_pi_S_B_F_B = full_pi_S_B_F_B

if full_pi_S_B_S_B == 0:
  full_pi_S_B_S_B = 1
else:
  full_pi_S_B_S_B = full_pi_S_B_S_B

if full_pi_S_B_F_E == 0:
  full_pi_S_B_F_E = 1
else:
  full_pi_S_B_F_E = full_pi_S_B_F_E

if full_pi_S_B_S_E == 0:
  full_pi_S_B_S_E = 1
else:
  full_pi_S_B_S_E = full_pi_S_B_S_E

if full_pi_F_E_F_B == 0:
  full_pi_F_E_F_B = 1
else:
  full_pi_F_E_F_B = full_pi_F_E_F_B

if full_pi_F_E_S_B == 0:
  full_pi_F_E_S_B = 1
else:
  full_pi_F_E_S_B = full_pi_F_E_S_B

if full_pi_F_E_F_E == 0:
  full_pi_F_E_F_E = 1
else:
  full_pi_F_E_F_E = full_pi_F_E_F_E

if full_pi_F_E_S_E == 0:
  full_pi_F_E_S_E = 1
else:
  full_pi_F_E_S_E = full_pi_F_E_S_E

if full_pi_S_E_F_B == 0:
  full_pi_S_E_F_B = 1
else:
  full_pi_S_E_F_B = full_pi_S_E_F_B

if full_pi_S_E_S_B == 0:
  full_pi_S_E_S_B = 1
else:
  full_pi_S_E_S_B = full_pi_S_E_S_B

if full_pi_S_E_F_E == 0:
  full_pi_S_E_F_E = 1
else:
  full_pi_S_E_F_E = full_pi_S_E_F_E

if full_pi_S_E_S_E == 0:
  full_pi_S_E_S_E = 1
else:
  full_pi_S_E_S_E = full_pi_S_E_S_E

#Kaufentscheidung - Tabellen

In [ ]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b> </b>','<b>Full Buy</b>','<b>Semi Buy</b>','<b>Full Eco</b>','<b>Semi Eco</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['Full Buy', 'Semi Buy', 'Full Eco', 'Semi Eco'],
      [pi_F_B_F_B.shape[0], pi_S_B_F_B.shape[0], pi_F_E_F_B.shape[0], pi_S_E_F_B.shape[0]],
      [pi_F_B_S_B.shape[0], pi_S_B_S_B.shape[0], pi_F_E_S_B.shape[0], pi_S_E_S_B.shape[0]],
      [pi_F_B_F_E.shape[0], pi_S_B_F_E.shape[0], pi_F_E_F_E.shape[0], pi_S_E_F_E.shape[0]],
      [pi_F_B_S_E.shape[0], pi_S_B_S_E.shape[0], pi_F_E_S_E.shape[0], pi_S_E_S_E.shape[0]]],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

print("Siege der T Seite")
fig.show()

Siege der T Seite


In [ ]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b> </b>','<b>Full Buy</b>','<b>Semi Buy</b>','<b>Full Eco</b>','<b>Semi Eco</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['Full Buy', 'Semi Buy', 'Full Eco', 'Semi Eco'],
      [CT_pi_F_B_F_B.shape[0], CT_pi_S_B_F_B.shape[0], CT_pi_F_E_F_B.shape[0], CT_pi_S_E_F_B.shape[0]],
      [CT_pi_F_B_S_B.shape[0], CT_pi_S_B_S_B.shape[0], CT_pi_F_E_S_B.shape[0], CT_pi_S_E_S_B.shape[0]],
      [CT_pi_F_B_F_E.shape[0], CT_pi_S_B_F_E.shape[0], CT_pi_F_E_F_E.shape[0], CT_pi_S_E_F_E.shape[0]],
      [CT_pi_F_B_S_E.shape[0], CT_pi_S_B_S_E.shape[0], CT_pi_F_E_S_E.shape[0], CT_pi_S_E_S_E.shape[0]]],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

print("Siege der CT Seite")
fig.show()

Siege der CT Seite


#Kaufentscheidung - Nash 

In [ ]:
#Payoff-Matrix für Spieler A, in diesem Fall die T-Seite
A = np.array([[pi_F_B_F_B.shape[0]/full_pi_F_B_F_B, pi_S_B_F_B.shape[0]/full_pi_S_B_F_B,  pi_F_E_F_B.shape[0]/full_pi_F_E_F_B,  pi_S_E_F_B.shape[0]/full_pi_S_E_F_B],
              [pi_F_B_S_B.shape[0]/full_pi_F_B_S_B, pi_S_B_S_B.shape[0]/full_pi_S_B_S_B,  pi_F_E_S_B.shape[0]/full_pi_F_E_S_B,  pi_S_E_S_B.shape[0]/full_pi_S_E_S_B],
              [pi_F_B_F_E.shape[0]/full_pi_F_B_F_E, pi_S_B_F_E.shape[0]/full_pi_S_B_F_E,  pi_F_E_F_E.shape[0]/full_pi_F_E_F_E,  pi_S_E_F_E.shape[0]/full_pi_S_E_F_E],
              [pi_F_B_S_E.shape[0]/full_pi_F_B_S_E, pi_S_B_S_E.shape[0]/full_pi_S_B_S_E,  pi_F_E_S_E.shape[0]/full_pi_F_E_S_E,  pi_S_E_S_E.shape[0]/full_pi_S_E_S_E]])
A = A.transpose()

#Payoff-Matrix für Spieler B, in diesem Fall die CT-Seite
B = np.array([[CT_pi_F_B_F_B.shape[0]/full_pi_F_B_F_B, CT_pi_S_B_F_B.shape[0]/full_pi_S_B_F_B, CT_pi_F_E_F_B.shape[0]/full_pi_F_E_F_B, CT_pi_S_E_F_B.shape[0]/full_pi_S_E_F_B],
              [CT_pi_F_B_S_B.shape[0]/full_pi_F_B_S_B, CT_pi_S_B_S_B.shape[0]/full_pi_S_B_S_B, CT_pi_F_E_S_B.shape[0]/full_pi_F_E_S_B, CT_pi_S_E_S_B.shape[0]/full_pi_S_E_S_B],
              [CT_pi_F_B_F_E.shape[0]/full_pi_F_B_F_E, CT_pi_S_B_F_E.shape[0]/full_pi_S_B_F_E, CT_pi_F_E_F_E.shape[0]/full_pi_F_E_F_E, CT_pi_S_E_F_E.shape[0]/full_pi_S_E_F_E],
              [CT_pi_F_B_S_E.shape[0]/full_pi_F_B_S_E, CT_pi_S_B_S_E.shape[0]/full_pi_S_B_S_E, CT_pi_F_E_S_E.shape[0]/full_pi_F_E_S_E, CT_pi_S_E_S_E.shape[0]/full_pi_S_E_S_E]])
B = B.transpose()

#Erstelle ein Spiel
CSGO_2 = nash.Game(A, B)

#Löse das Spiel und gebe das Ergebnis aus
equilibria = CSGO_2.support_enumeration()
for eq in equilibria:
    print(eq)